# Tests
Environment tests for python libraries.

```
python -m venv .venv
.\.venv\Scripts\activate
```

```
pip install ipykernel
ipython kernel install --user --name=.venv
```

```
pip install nbformat
```

## Numpy

```
pip install numpy
```

In [ ]:
import numpy as np

def jit_fft(x):
    return np.fft.fft(x)

a = np.array([1, 6, 1, 8, 0, 3, 3, 9])
jit_fft(a)

## Numba (cpu)
[Requires installing rocket-fft, but installing rocket-fft might take care of installing numba.]

```
pip install numba
pip install rocket-fft
```

In [ ]:
import numba as nb
import numpy as np

@nb.njit
def jit_fft(x):
    return np.fft.fft(x)

a = np.array([1, 6, 1, 8, 0, 3, 3, 9])
jit_fft(a)

## Numba (gpu)

```
pip install cupy
```

In [ ]:
import numpy as np
import cupy as cp
import numba as nb
from numba import cuda

@cuda.jit
def custom_fft(A, Ak):
    # Perform operations on Ak (modify Ak as needed)
    # Example: Ak *= 2

    # Compute FFT of Ak using CuPy
    Ak_fft = cp.fft.fft2(Ak)

    # Perform additional operations on Ak_fft
    # Example: Ak_fft += 1

    # Compute inverse FFT to get A
    A[:] = cp.fft.ifft2(Ak_fft)

if __name__ == "__main__":
    A = np.random.rand(4, 4)
    Ak = np.random.rand(4, 4)

    # Transfer arrays to GPU
    A_gpu = cp.asarray(A)
    Ak_gpu = cp.asarray(Ak)

    # Call the custom kernel
    custom_fft, 1

    # Transfer result back to CPU
    A_result = cp.asnumpy(A_gpu)

    print("Modified A:")
    print(A_result)

## iPyWidgets

```
pip install ipywidgets
```

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
slider = widgets.FloatSlider(
    value=7,
    min=0,
    max=10,
    step=0.1,
    description='Number:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.2f'
)


In [ ]:
display(slider)

In [ ]:
# Access the slider value
def on_value_change(change):
    print(f'Slider value: {change["new"]}')

slider.observe(on_value_change, names='value')

### Plotting example

In [ ]:
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact

fig = go.FigureWidget()
fig.add_scatter()

xs=np.linspace(0, 6, 100)

@interact(a=(1.0, 4.0, 0.01), b=(0, 10.0, 0.01), color=['red', 'green', 'blue'])
def update(a=3.6, b=4.3, color='blue'):
    with fig.batch_update():
        fig.data[0].x=xs
        fig.data[0].y=np.sin(a*xs-b)
        fig.data[0].line.color=color

fig


#### Dash

```
pip install dash
```

In [ ]:
import plotly.express as px
df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Animated GDP and population over decades'),
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=["GDP - Scatter", "Population - Bar"],
        value='GDP - Scatter',
    ),
    dcc.Loading(dcc.Graph(id="graph"), type="cube")
])


@app.callback(
    Output("graph", "figure"), 
    Input("selection", "value"))
def display_animated_graph(selection):
    df = px.data.gapminder() # replace with your own data source
    animations = {
        'GDP - Scatter': px.scatter(
            df, x="gdpPercap", y="lifeExp", animation_frame="year", 
            animation_group="country", size="pop", color="continent", 
            hover_name="country", log_x=True, size_max=55, 
            range_x=[100,100000], range_y=[25,90]),
        'Population - Bar': px.bar(
            df, x="continent", y="pop", color="continent", 
            animation_frame="year", animation_group="country", 
            range_y=[0,4000000000]),
    }
    return animations[selection]


app.run_server(debug=True)